In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sns
%matplotlib inline
import pulp

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('nutrition.csv').drop('Unnamed: 0',axis=1)

In [6]:
df = df[['name', 'calories', 'total_fat', 'protein', 'water', 'carbohydrate', 'fiber']]

In [7]:
lists = ['protein', 'water', 'carbohydrate',
       'fiber']

for col in lists:
    df[col] = df[col].str.replace(' g', '', regex=False)
    df[col] = df[col].astype('float')

In [8]:
df['total_fat'] = df['total_fat'].str.replace('g', '', regex=False)

df['total_fat'] = df['total_fat'].astype('float')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8789 entries, 0 to 8788
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          8789 non-null   object 
 1   calories      8789 non-null   int64  
 2   total_fat     8789 non-null   float64
 3   protein       8789 non-null   float64
 4   water         8789 non-null   float64
 5   carbohydrate  8789 non-null   float64
 6   fiber         8789 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 480.8+ KB


In [10]:
df.head()

,name,calories,total_fat,protein,water,carbohydrate,fiber
0,Cornstarch,381,0.1,0.26,8.32,91.27,0.9
1,"Nuts, pecans",691,72.0,9.17,3.52,13.86,9.6
2,"Eggplant, raw",25,0.2,0.98,92.30,5.88,3.0
3,"Teff, uncooked",367,2.4,13.30,8.82,73.13,8.0
4,"Sherbet, orange",144,2.0,1.10,66.10,30.40,1.3


In [12]:
week_days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
split_values = np.linspace(0,len(df),8).astype(int)
split_values[-1] = split_values[-1]-1
def random_dataset():
    frac_data = df.sample(frac=1).reset_index().drop('index',axis=1)
    day_data = []
    for s in range(len(split_values)-1):
        day_data.append(frac_data.loc[split_values[s]:split_values[s+1]])
    return dict(zip(week_days,day_data))

In [13]:
random_dataset()['Monday']

,name,calories,total_fat,protein,water,carbohydrate,fiber
0,"McDONALD'S, Hash Brown",271,16.0,2.48,50.42,28.56,2.8
1,"Babyfood, junior, apricot with tapioca, fruit",63,0.0,0.30,82.10,17.30,1.5
2,"Waffles, toasted, ready -to-heat, frozen, plain",312,9.6,7.19,30.59,49.29,2.4
3,"Chicken, simmered, cooked, giblets, capons",164,5.4,26.39,66.54,0.76,0.0
4,"Peanut flour, low fat",428,22.0,33.80,7.80,31.27,15.8
...,...,...,...,...,...,...,...
1251,"Dessert topping, frozen, semi solid",318,25.0,1.25,50.21,23.05,0.0
1252,"Cereals ready-to-eat, Chocolate, CHEERIOS, GEN...",380,5.1,5.80,2.20,83.50,6.2
1253,"Beef, grilled, cooked, choice, trimmed to 0"" f...",283,20.0,24.95,54.74,0.00,0.0
1254,"Cereals ready-to-eat, KELLOGG'S SPECIAL K Low ...",375,5.4,13.10,3.50,74.80,11.3


In [14]:
def build_nutritional_values(kg,calories):
    protein_calories = kg*4
    res_calories = calories-protein_calories
    carb_calories = calories/2.
    fat_calories = calories-carb_calories-protein_calories
    res = {'Protein Calories':protein_calories,'Carbohydrates Calories':carb_calories,'Fat Calories':fat_calories}
    return res

In [15]:
def extract_gram(table):
    protein_grams = table['Protein Calories']/4.
    carbs_grams = table['Carbohydrates Calories']/4.
    fat_grams = table['Fat Calories']/9.
    res = {'Protein Grams':protein_grams, 'Carbohydrates Grams':carbs_grams,'Fat Grams':fat_grams}
    return res

In [16]:
build_nutritional_values(70,2000)

{'Protein Calories': 280,
 'Carbohydrates Calories': 1000.0,
 'Fat Calories': 720.0}

In [17]:
print(extract_gram(build_nutritional_values(70,2000)))

{'Protein Grams': 70.0, 'Carbohydrates Grams': 250.0, 'Fat Grams': 80.0}


In [23]:
days_data = random_dataset()
def model(day,kg,calories):
    G = extract_gram(build_nutritional_values(kg,calories))
    E = G['Carbohydrates Grams']
    F = G['Fat Grams']
    P = G['Protein Grams']
    day_data = days_data[day]
    day_data = day_data[day_data.calories!=0]
    food = day_data.name.tolist()
    c  = day_data.calories.tolist()
    x  = pulp.LpVariable.dicts( "x", indices = food, lowBound=0, upBound=1.5, cat='Continuous', indexStart=[] )
    e = day_data.carbohydrate.tolist()
    f = day_data.total_fat.tolist()
    p = day_data.protein.tolist()
    prob  = pulp.LpProblem( "Diet", pulp.LpMinimize )
    prob += pulp.lpSum( [x[food[i]]*c[i] for i in range(len(food))]  )
    prob += pulp.lpSum( [x[food[i]]*e[i] for i in range(len(x)) ] )>=E
    prob += pulp.lpSum( [x[food[i]]*f[i] for i in range(len(x)) ] )>=F
    prob += pulp.lpSum( [x[food[i]]*p[i] for i in range(len(x)) ] )>=P
    prob.solve()
    variables = []
    values = []
    for v in prob.variables():
        variable = v.name
        value = v.varValue
        variables.append(variable)
        values.append(value)
    values = np.array(values).round(2).astype(float)
    sol = pd.DataFrame(np.array([food,values]).T, columns = ['Food','Quantity'])
    sol['Quantity'] = sol.Quantity.astype(float)
    return sol

In [24]:
sol_monday = model('Monday',70,1500)

In [25]:
sol_monday

,Food,Quantity
0,"Sausage, beef, pork, Berliner",0.0
1,"Lamb, raw, trimmed to 1/8"" fat, separable lean...",0.0
2,"KASHI, Oatmeal Raisin Flax Cookies, TLC",0.0
3,"KASHI, Frozen Entree, Sesame Chicken, STEAM MEAL",0.0
4,"Milk, non-soy, imitation",0.0
...,...,...
1244,"Chicken breast, sliced, mesquite flavor, fat-free",0.0
1245,"Cereals, dry, plain, original, MALT-O-MEAL",0.0
1246,"Pork, roasted, cooked, boneless, separable lea...",0.0
1247,"Pie, Commercially Prepared, Dutch Apple",0.0


In [28]:
def model(prob,day,kg,calories):
    G = extract_gram(build_nutritional_values(kg,calories))
    E = G['Carbohydrates Grams']
    F = G['Fat Grams']
    P = G['Protein Grams']
    day_data = days_data[day]
    day_data = day_data[day_data.calories!=0]
    food = day_data.name.tolist()
    c  = day_data.calories.tolist()
    x  = pulp.LpVariable.dicts( "x", indices = food, lowBound=0, upBound=1.5, cat='Continuous', indexStart=[] )
    e = day_data.carbohydrate.tolist()
    f = day_data.total_fat.tolist()
    p = day_data.protein.tolist()
#    prob  = pulp.LpProblem( "Diet", LpMinimize )
    prob += pulp.lpSum( [x[food[i]]*c[i] for i in range(len(food))]  )
    prob += pulp.lpSum( [x[food[i]]*e[i] for i in range(len(x)) ] )>=E
    prob += pulp.lpSum( [x[food[i]]*f[i] for i in range(len(x)) ] )>=F
    prob += pulp.lpSum( [x[food[i]]*p[i] for i in range(len(x)) ] )>=P
    prob.solve()
    variables = []
    values = []
    for v in prob.variables():
        variable = v.name
        value = v.varValue
        variables.append(variable)
        values.append(value)
    values = np.array(values).round(2).astype(float)
    sol = pd.DataFrame(np.array([food,values]).T, columns = ['Food','Quantity'])
    sol['Quantity'] = sol.Quantity.astype(float)
    sol = sol[sol['Quantity']!=0.0]
    sol.Quantity = sol.Quantity*100
    sol = sol.rename(columns={'Quantity':'Quantity (g)'})
    return sol
def total_model(kg,calories):
    result = []
    for day in week_days:
        prob  = pulp.LpProblem( "Diet", pulp.LpMinimize )
        print('Building a model for day %s \n'%(day))
        result.append(model(prob,day,kg,calories))
    return dict(zip(week_days,result))

In [41]:
diet = total_model(70,2000)

Building a model for day Monday 

Building a model for day Tuesday 

Building a model for day Wednesday 

Building a model for day Thursday 

Building a model for day Friday 

Building a model for day Saturday 

Building a model for day Sunday 



In [42]:
diet

{'Monday':                                                    Food  Quantity (g)
 52                   Pie, prepared from recipe, pumpkin         150.0
 72    Brussels sprouts, without salt, drained, boile...         150.0
 243             Salad dressing, regular, ranch dressing         150.0
 348        POPEYES, analyzed 2006, Spicy Chicken Strips         150.0
 499   Beef, broiled, cooked, all grades, trimmed to ...         150.0
 732   Beverages, ready-to-drink., G performance O 2,...          61.0
 1057             Alcoholic beverage, white, table, wine          31.0
 1137            Veal, raw, separable lean only, sirloin         150.0
 1218                                      Pepeao, dried          40.0,
 'Tuesday':                                                    Food  Quantity (g)
 63    Beef, roasted, cooked, all grades, trimmed to ...         144.0
 629   Beef, raw, all grades, trimmed to 0" fat, sepa...         150.0
 811        Gums, seed gums (includes locust bean, guar

In [34]:
split_size = np.linspace(0,len(df),8)